### 초기 설정

In [34]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [35]:
!pip install efficientnet_pytorch

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [36]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os
import logging
import torch
import pickle
from tqdm import tqdm # progress bar
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [37]:
# dataset path
data_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/test/"
# cluster_list.txt path
cluster_list_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/cluster_list.txt"
# feature path
feat_path = "/content/drive/MyDrive/Colab/Sketch_RNN_Together/feature/features_2500.npy"
# filename path
filename_path = "/content/drive/MyDrive/Colab/Sketch_RNN_Together/feature/filenames_2500.npy"
# textfile path
textfile_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/result.txt"

In [38]:
# make data_list : .npy files
os.chdir(data_path) # change directory path
data_list = []
with os.scandir(data_path) as files:
    for file in files:
        if file.name.endswith('.npy'):
            data_list.append(file.name)
# make cluster_list : cluster label name files
cluster_list = open(cluster_list_path , 'r').read().split('\n')
# make option_list
option_list = {'feature':'features_hw_2500'}

### Feature Extraction

In [ ]:
# transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),]) # compose image (resize, totensor, normalize)

def ExtractFeature(image, model):
    img = tfms(Image.fromarray(image.reshape(28,28)).convert("RGB")).unsqueeze(0) # unsqueeze: vector to tensor
    features = model.extract_features(img)
    return features

In [ ]:
#efficientNet
#feature extraction
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {} # {} set
for npy in tqdm(data_list):
    image_list = np.load(npy)
    for i, image in enumerate(image_list[:1000]):
      feat = ExtractFeature(image, model)
      feat = feat.detach().numpy().reshape(-1) # detach: copy tensor without gradient
      name = npy.split('.')[0] + '_' + str(i)
      data[name] = (feat) 
      if(i % 200 == 0):
        np.save(feat_path, np.array(list(data.values())))
        np.save(filename_path, np.array(list(data.keys())))

np.save(feat_path, np.array(list(data.values()))) # get a list of the filenames
np.save(filename_path, np.array(list(data.keys()))) # get a list of just the features


  0%|          | 0/10 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0



100%|██████████| 10/10 [33:11<00:00, 199.13s/it]


### Dimension Reduction

In [39]:
feat = np.load(feat_path)

In [ ]:
# PCA
from sklearn.decomposition import PCA
option_list['reduction'] = {'method':'PCA', 'n_components':2}
pca = PCA(n_components=option_list['reduction']['n_components'], random_state=0)
pca.fit(feat)
x = pca.transform(feat)

In [ ]:
# TSNE
from sklearn.manifold import TSNE
option_list['reduction'] = {'method':'TSNE', 'n_components':2}
tsne = TSNE(n_components=option_list['reduction']['n_components'], init='pca', random_state=0)
x = tsne.fit_transform(feat)

### Clustering

In [ ]:
def Cluster(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    option_list['clustering'] = {'method':cmd}
    kmeans = KMeans(n_clusters = cluster_num, n_jobs = -1, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_
  elif cmd == 'affinity':
    from sklearn.cluster import AffinityPropagation
    option_list['clustering'] = {'method':cmd, 'preference':-50}
    affinity = AffinityPropagation(preference=option_list['clustering']['preference'])
    affinity.fit(input)

In [ ]:
labels = Cluster('kmeans', len(cluster_list), x)
import plotly.express as px
from plotly.offline import plot
fig = px.scatter(x, x=0, y=1, color = labels)
plot(fig)

### Testing

In [ ]:
filenames = np.load(filename_path)
groups = {}
# holds the cluster id and the images { id: [images] }
for f, cluster in zip(filenames, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(f)
    else:
        groups[cluster].append(f)

# make cluster_dict for calculate acc
print('cluster_list')
print(cluster_list)
print()
cluster_dict = {}
print('cluster_dict')
for cluster in groups:
  image_count = []
  image_name = []
  for image in groups[cluster]:
    image_name.append(image.split('_')[3])
  for name in cluster_list:
    image_count.append(image_name.count(name))

  cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))] # Select the most frequently classified label as the correct label
print(sorted(cluster_dict.items()))
print()

# Accuracy
from sklearn.metrics import f1_score
pred = []
gt = []    
for cluster in groups:
    for category in groups[cluster]:
        pred.append(cluster_dict[cluster])
        gt.append(category.split('_')[3])

print(str(f1_score(gt, pred, average='micro') * 100) + '%')

# Write
with open(textfile_path, 'a', encoding='UTF-8') as f:
  for key, value in option_list.items():
    f.write(f'{value} ')
f.write(f'\n')